In [1]:
import sys
from pathlib import Path

# Add the src directory to Python path
repo_root = Path.cwd().parent  # Go up one level from notebooks/ to repo root
src_path = repo_root / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Now import the class
from models.basic import TwoStageCapacityAndProcurementPlanning

print(f"✓ Successfully imported TwoStageCapacityAndProcurementPlanning")
print(f"  Repository root: {repo_root}")
print(f"  Source path: {src_path}")

✓ Successfully imported TwoStageCapacityAndProcurementPlanning
  Repository root: c:\Users\siatr\VS Code Workspace\steel-industry-stochastic-optimization
  Source path: c:\Users\siatr\VS Code Workspace\steel-industry-stochastic-optimization\src


Load Data from Federal Reserve Economic Data (FRED) API
- IPG3311A2S: seasonally adjusted industrial production index for steel works and blast furnaces (NAICS code 3311). It measures the output of steel manufacturing facilities in the United States
- WPU1012: Producer Price Index (PPI) for iron and steel scrap (commodity code 1012). It measures the average price changes for scrap metal used in steel production, indexed (1982=100) and updated monthly by the Bureau of Labor Statistics.
- 

In [ ]:
# Load data
data = TwoStageCapacityAndProcurementPlanning.load_data_from_fredapi(
    'add_fred_key_here',  # Replace with your actual FRED API key
    steel_demand_identifier='IPG3311A2S', 
    scrap_price_identifier='WPU1012', 
    steel_price_identifier='WPU101704',    
    plot_data=True)

In [ ]:
data_subset = TwoStageCapacityAndProcurementPlanning.get_n_observations(
    data,
    n=180,
    p=2,
    last_observation='2019-12-01',
    plot_data=True)

In [ ]:
Δlog = TwoStageCapacityAndProcurementPlanning.log_returns(
    data_subset,
    plot_data=True,
    print_stats=True)

print(f"✓ Log returns computed: {Δlog.shape} observations")
print(f"Missing values: {Δlog.isnull().sum().sum()}")

In [ ]:
var_model = TwoStageCapacityAndProcurementPlanning.fit_VAR_model(
    data=data_subset,
    p=2,  # Let the function select optimal lag order
    # NOTE: Testing options can be adjusted as needed, the objective is to ensure model follows real economy dynamics
    testing=['stability', 'irf', 'corr', 'sim_stats', 'residual_tests'],
    method='bic',
    print_warnings=True
)

In [ ]:
shock_distribution_analysis_results = TwoStageCapacityAndProcurementPlanning.analyze_shock_distributions(var_model)

In [ ]:
scenario_returns, prob = TwoStageCapacityAndProcurementPlanning.generate_future_returns_scenarios(
    var_model=var_model, 
    simulation_start_date=data_subset.index.max().strftime('%Y-%m'),
    horizon=24,
    n_scenarios=1000,
    seed=42,
    shock_distribution='t',
    distribution_params={'df':7})

In [ ]:
real_prices = {
    'P': 800,    # €/ton steel price
    'C': 400,    # €/ton scrap cost  
    'D': 50_000   # tons/month demand
}
scenario_levels, info = TwoStageCapacityAndProcurementPlanning.reconstruct_levels_from_returns(
    scenario_returns=scenario_returns,
    historical_data=data_subset,
    anchor_date=None,
    real_prices=real_prices
)

print(f"Steel prices range: €{scenario_levels['P'].min():.0f} - €{scenario_levels['P'].max():.0f}/ton")
print(f"Demand range: {scenario_levels['D'].min():,.0f} - {scenario_levels['D'].max():,.0f} tons/month")

In [ ]:
# Backtesting with actual trajectory
actual_future = data.loc['2020-01-01':'2021-12-01']
fig, axes = TwoStageCapacityAndProcurementPlanning.plot_scenarios_evolution(
    scenarios=scenario_levels,
    historical_data=data_subset,
    max_history=180,
    max_number_of_scenarios=50,
    prob=prob,
    future_trajectory=actual_future,
    title_prefix="Backtest - "
)

In [ ]:
# With stress scenarios and diagnostics
scenarios_red, prob_red = TwoStageCapacityAndProcurementPlanning.reduce_scenarios_kmedoids(
    scenarios=scenario_levels,
    prob=prob,
    n_scenario_clusters=100,
    stress_pct=0.01,
    stress_direction='both'
)

In [ ]:
fig, axes = TwoStageCapacityAndProcurementPlanning.plot_scenarios_evolution(
    scenarios=scenarios_red,
    historical_data=data_subset,
    max_history=30,
    max_number_of_scenarios=100,
    prob=prob_red,
    future_trajectory=actual_future,
    title_prefix="Backtest - "
)

In [ ]:
decisions = TwoStageCapacityAndProcurementPlanning.optimize_capacity_and_procurement(
    scenarios=scenarios_red,
    prob=prob_red,
    alpha=1.01,  # scrap-to-product ratio
    c_var=250.0, # variable production cost €/ton
    c_cap_base=10.0, # fixed capacity cost €/ton capacity
    c_cap_flex=30.0, # flexible capacity cost €/ton capacity
    delta_base=5.0, # base scrap procurement premium €/ton
    delta_spot=15.0, # spot scrap procurement premium €/ton
    pen_unmet=0.0, # penalty cost for unmet demand €/ton
    gamma_cap=0.3, # capacity adjustment cost factor
    gamma_scrap=0.8, # scrap procurement adjustment cost factor
    solver="highs",
    return_full_results=False
)

print("Optimal capacity decisions:")
decisions

In [ ]:
import warnings
import pandas as pd

# Suppress specific pandas warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings('ignore', message='.*DataFrame.stack.*')

# After optimization
profit_analysis = TwoStageCapacityAndProcurementPlanning.plot_profit_distribution_over_time(
    decisions=decisions,
    scenarios=scenarios_red,
    prob=prob_red,
    alpha=1.01,  # scrap-to-product ratio
    c_var=250.0, # variable production cost €/ton
    c_cap_base=10.0, # fixed capacity cost €/ton capacity
    c_cap_flex=30.0, # flexible capacity cost €/ton capacity
    delta_base=5.0, # base scrap procurement premium €/ton
    delta_spot=15.0, # spot scrap procurement premium €/ton
    pen_unmet=0.0, # penalty cost for unmet demand €/ton
    gamma_cap=0.3, # capacity adjustment cost factor
    gamma_scrap=0.8, # scrap procurement adjustment cost factor
    confidence_levels=[0.01, 0.99],  # 98% CI
)

print(f"Expected total profit: €{profit_analysis['cumulative_stats']['expected_cumulative']:,.0f}")

In [ ]:
# Run backtesting simulation
actual_future = data.loc['2020-01-01':'2021-12-01']
backtest_results = TwoStageCapacityAndProcurementPlanning.backtesting_simulation(
    decisions=decisions,
    actual_future_data=actual_future,
    alpha=1.01,  # scrap-to-product ratio
    c_var=250.0, # variable production cost €/ton
    c_cap_base=10.0, # fixed capacity cost €/ton capacity
    c_cap_flex=30.0, # flexible capacity cost €/ton capacity
    delta_base=5.0, # base scrap procurement premium €/ton
    delta_spot=15.0, # spot scrap procurement premium €/ton
    pen_unmet=0.0, # penalty cost for unmet demand €/ton
    gamma_cap=0.3, # capacity adjustment cost factor
    gamma_scrap=0.8, # scrap procurement adjustment cost factor
    real_prices={'P': 800, 'C': 400, 'D': 50000}
)

print(f"Total profit: €{backtest_results['summary_stats']['total_profit']:,.0f}")

## Test CVaR Risk-Averse Optimization

Now let's test the new CVaR functionality to see how risk aversion affects capacity decisions.

In [ ]:
decisions_risk_neutral = TwoStageCapacityAndProcurementPlanning.optimize_capacity_and_procurement(
    scenarios=scenarios_red,
    prob=prob_red,
    alpha=1.01,  # scrap-to-product ratio
    c_var=250.0, # variable production cost €/ton
    c_cap_base=10.0, # fixed capacity cost €/ton capacity
    c_cap_flex=30.0, # flexible capacity cost €/ton capacity
    delta_base=5.0, # base scrap procurement premium €/ton
    delta_spot=15.0, # spot scrap procurement premium €/ton
    pen_unmet=0.0, # penalty cost for unmet demand €/ton
    gamma_cap=0.3, # capacity adjustment cost factor
    gamma_scrap=0.8, # scrap procurement adjustment cost factor
    solver="highs",
    return_full_results=True,
    risk_aversion=0.0,  # Risk-neutral
    cvar_alpha=0.05,
    log_level="WARNING"  # Standard logging - shows major steps only
)

print("\nRisk-neutral decisions:")
print(f"Average base capacity: {decisions_risk_neutral['q_cap_base'].mean():.1f} tons/month")
print(f"Expected profit: €{decisions_risk_neutral['expected_profit']:,.0f}")

In [ ]:
# Test 2: Moderate risk aversion (λ=0.3)
print("\n" + "="*60)
print("TEST 2: MODERATE RISK AVERSION (λ=0.3)")
print("="*60)

decisions_moderate_risk = TwoStageCapacityAndProcurementPlanning.optimize_capacity_and_procurement(
    scenarios=scenarios_red,
    prob=prob_red,
    alpha=1.01,
    c_var=250.0,
    c_cap_base=10.0,
    c_cap_flex=30.0,
    delta_base=5.0,
    delta_spot=15.0,
    pen_unmet=0.0,
    gamma_cap=0.3,
    gamma_scrap=0.8,
    solver="highs",
    return_full_results=True,
    risk_aversion=0.3,  # Moderate risk aversion
    cvar_alpha=0.05
)

print("\nModerate risk aversion decisions:")
print(f"Average base capacity: {decisions_moderate_risk['q_cap_base'].mean():.1f} tons/month")
print(f"Expected profit: €{decisions_moderate_risk['expected_profit']:,.0f}")
if 'risk_metrics' in decisions_moderate_risk:
    print(f"VaR: €{decisions_moderate_risk['risk_metrics']['VaR']:,.0f}")
    print(f"CVaR (worst 5%): €{decisions_moderate_risk['risk_metrics']['CVaR']:,.0f}")